# Análise de acidentes fatais na estrada na Austrália entre 1989 e 2021


In [122]:
#TODO DESCOMENTAR PARA ENTREGA
# pd.set_option('display.max_columns', None)
# path_to_arthurs_drive = '/content/drive/MyDrive/IA - P3/Crash_Data.csv'
# df = pd.read_csv(path_to_arthurs_drive)

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
# from sklearn.impute import KNNImputer
# from matplotlib.dates import DateFormatter

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("Crash_Data.csv")


# 1. Visualizando os dados brutos

In [117]:
df.head(5)

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,...,Age,National Remoteness Areas,SA4 Name 2016,National LGA Name 2017,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day
0,20212133,Vic,9,2021,Sunday,0:30,Single,NaN,NaN,NaN,...,38,Inner Regional Australia,Melbourne - Outer East,Yarra Ranges (S),Arterial Road,No,No,26_to_39,Weekend,Night
1,20214022,SA,9,2021,Saturday,23:31,Multiple,No,No,No,...,28,Major Cities of Australia,Adelaide - North,Playford (C),NaN,No,No,26_to_39,Weekend,Night
2,20212096,Vic,9,2021,Saturday,23:00,Single,NaN,NaN,NaN,...,19,Inner Regional Australia,Hume,Wangaratta (RC),Access road,No,No,17_to_25,Weekend,Night
3,20212145,Vic,9,2021,Saturday,22:25,Single,NaN,NaN,NaN,...,23,Outer Regional Australia,Hume,Wangaratta (RC),Arterial Road,No,No,17_to_25,Weekend,Night
4,20212075,Vic,9,2021,Saturday,5:15,Single,NaN,NaN,NaN,...,46,Major Cities of Australia,Melbourne - South East,Casey (C),Local Road,No,No,40_to_64,Weekend,Night


In [118]:
# Emite um relatório simples sobre TODAS AS COLUNAS do Data frame
for column in df:
  soma_valores_nulos = df[column].isnull().sum()
  total_valores = len(df[column])
  porcentagem_nulos = round((soma_valores_nulos / total_valores) * 100, 2)

  print(f'Coluna {column} possui {soma_valores_nulos} valores nulos do total de {total_valores} -> {porcentagem_nulos}% \n valores diferentes: {set(df[column])} \n\n')


Coluna Crash ID possui 0 valores nulos do total de 52843 -> 0.0% 
 valores diferentes: {20185088, 20185089, 20185090, 20185091, 20185092, 20185093, 20185094, 20185095, 20185096, 20185097, 20185098, 20185099, 20185100, 20185101, 20185102, 20185103, 20185104, 20185105, 20185106, 20185107, 20185108, 20185109, 20185110, 20185111, 20185112, 20185113, 20185114, 20185115, 20185116, 20185117, 20185118, 20185119, 20185120, 20185121, 20185122, 20185123, 20185124, 20185125, 20185126, 20185127, 20185128, 20185129, 20185130, 20185131, 20185132, 20185133, 20185134, 20185135, 20185136, 20185137, 20185138, 20185139, 20185140, 20185141, 20185142, 20185143, 20185144, 20185145, 20054068, 20054070, 20054067, 20054076, 20054075, 20054071, 20054080, 20054079, 20054073, 20054077, 20054082, 20054085, 20054086, 20054087, 20054084, 20054089, 20054090, 20054091, 20054092, 20054093, 20054094, 20054095, 20054096, 20054097, 20054098, 20054099, 20054100, 20054101, 20054102, 20054103, 20054104, 20054105, 20054106, 20

# 2. Preparando dados para análise

In [119]:
# Separa as colunas que serão usadas para análise e descarta as demais
df = df[[
  'Time',
  'Time of day',
  'Day of week',
  'Month',
  'Year',
  'Dayweek',
  'Crash Type',
  'Road User',
  'Gender',
  'Age',
  'Age Group',
  'State',
  'Speed Limit',
  'Bus Involvement',
  'Articulated Truck Involvement',
  'Christmas Period',
  'Easter Period',
]]


In [120]:
df.head(5)

,Time,Time of day,Day of week,Month,Year,Dayweek,Crash Type,Road User,Gender,Age,Age Group,State,Speed Limit,Bus Involvement,Articulated Truck Involvement,Christmas Period,Easter Period
0,0:30,Night,Weekend,9,2021,Sunday,Single,Motorcycle rider,Male,38,26_to_39,Vic,NaN,NaN,NaN,No,No
1,23:31,Night,Weekend,9,2021,Saturday,Multiple,Pedestrian,Female,28,26_to_39,SA,110,No,No,No,No
2,23:00,Night,Weekend,9,2021,Saturday,Single,Passenger,Male,19,17_to_25,Vic,NaN,NaN,NaN,No,No
3,22:25,Night,Weekend,9,2021,Saturday,Single,Driver,Male,23,17_to_25,Vic,NaN,NaN,NaN,No,No
4,5:15,Night,Weekend,9,2021,Saturday,Single,Motorcycle rider,Male,46,40_to_64,Vic,NaN,NaN,NaN,No,No


In [121]:
# Drop nas linhas onde a existe ao menos um valor nulo
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51997 entries, 1 to 52842
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Time                           51997 non-null  object
 1   Time of day                    51997 non-null  object
 2   Day of week                    51997 non-null  object
 3   Month                          51997 non-null  int64 
 4   Year                           51997 non-null  int64 
 5   Dayweek                        51997 non-null  object
 6   Crash Type                     51997 non-null  object
 7   Road User                      51997 non-null  object
 8   Gender                         51997 non-null  object
 9   Age                            51997 non-null  int64 
 10  Age Group                      51997 non-null  object
 11  State                          51997 non-null  object
 12  Speed Limit                    51997 non-null  object
 13  Bus In

# Características da Análise para explicação na documentação/apresentação
#### 1. Aprendizado: Supervisionado
#### 2. Classificação: Multiclass Classification 